In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
ls

drive/  sample_data/


In [3]:
cd /content/drive/MyDrive/AI/W2 - RAG - ANS

/content/drive/MyDrive/AI/W2 - RAG - ANS


In [6]:
ls

'RAG - ANS.ipynb'


setup library

In [4]:
!pip install -q transformers==4.52.4
!pip install -q bitsandbytes==0.46.0
!pip install -q accelerate==1.7.0
!pip install -q langchain==0.3.25
!pip install -q langchainhub==0.1.21
!pip install -q langchain-chroma==0.2.4
!pip install -q langchain_experimental==0.3.4
!pip install -q langchain-community==0.3.24
!pip install -q langchain_huggingface==0.2.0
!pip install -q python-dotenv==1.1.0
!pip install -q pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 132.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done

we need a vector DB
-> convert an file pdf to DB

In [6]:
import torch

from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface.llms import HuggingFacePipeline

from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.chains import ConversationalRetrievalChain
from langchain_experimental.text_splitter import SemanticChunker

from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

In [7]:
Loader = PyPDFLoader

FILE_PATH = "/content/drive/MyDrive/AI/W2 - RAG - ANS/AI Agents in Action (Micheal Lanham) (Z-Library).pdf"

loader = Loader(FILE_PATH)
documents = loader.load()

Initialize embedding model
-> convert text to vector

In [8]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

all vectors will be used in **semantic chunking** process

In [9]:
# text spliter
semantic_splitter = SemanticChunker(
    embeddings=embeddings,
    buffer_size = 1, # how many sentence will be combine befor split -> 1 = each group has 1 sentence
    breakpoint_threshold_type = "percentile", # how to caculate point for make 1 section
    breakpoint_threshold_amount = 95, #
    min_chunk_size = 500,
    add_start_index = True # where last index of the splitter
)

# to improve the quality of chunk text we will ussing semantic chunker , split base on semantic not a size of paragrah


In [10]:
docs = semantic_splitter.split_documents(documents)

In [11]:
print("Number of semantic chunk: ",len(docs))

Number of semantic chunk:  615


Vector DB

In [12]:
vector_db = Chroma.from_documents(documents=docs, embedding = embeddings)
retriever = vector_db.as_retriever()

In [13]:
result = retriever.invoke("What is CrewAI ?")

In [14]:
print("Number of relevant documents: ", len(result))

Number of relevant documents:  4


In [15]:
print(result)

[Document(id='dca29b66-3376-42c6-9ca0-8494a7f15b42', metadata={'page': 120, 'author': 'Micheal Lanham', 'source': '/content/drive/MyDrive/AI/W2 - RAG - ANS/AI Agents in Action (Micheal Lanham) (Z-Library).pdf', 'icnappplatform': 'Windows', 'start_index': 0, 'creationdate': '2025-01-28T09:33:03+00:00', 'total_pages': 346, 'page_label': '97', 'producer': 'Acrobat Distiller 20.0 (Windows); modified using iText® Core 8.0.2 (AGPL version) ©2000-2023 Apryse Group NV', 'moddate': 'D:20250219072748', 'icnappname': 'Foxit Advanced PDF Editor', 'creator': 'FrameMaker 16.0.1(Foxit Advanced PDF Editor)', 'title': 'AI Agents in Action', 'icnappversion': '3.05'}, page_content='97Summary\n\uf0a1 CrewAI supports integration with observability tools such as AgentOps, which\nprovides insights into agent performance, interaction efficiency, and cost\nmanagement. \uf0a1 AgentOps is an agent observability platform that can help you easily monitor\nextensive agent interactions.'), Document(id='d96cbf60-581a

Iinitialize LLMs - Vicuna

In [16]:
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [17]:
MODEL_NAME = "lmsys/vicuna-7b-v1.5"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=nf4_config,
    low_cpu_mem_usage=True
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [23]:
# CN 14 Intergrate tokenizer and model to an pipeline for ease to use
model_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    max_new_tokens=512,
    pad_token_id=tokenizer.eos_token_id,
)

llm = HuggingFacePipeline(pipeline=model_pipeline)

Device set to use cuda:0


RUN PROGRAM

In [24]:
prompt  = hub.pull("rlm/rag-prompt")

def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


USER_QUESTION = 'What is CrewAI ?'

output = rag_chain.invoke(USER_QUESTION)
answer = output.split('Answer:')[1].strip()

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [25]:
print(answer)

CrewAI is an open-source multi-agent platform that allows users to build and train teams of agents to perform various tasks, such as coding games. It supports integration with observability tools like AgentOps, which provides insights into agent performance, interaction efficiency, and cost management. With CrewAI, users can create different types of agents, such as senior software engineers, QA engineers, and chief QA engineers, and assign them specific roles, goals, and backstories. The platform also allows for hierarchical processing and the addition of custom packages like AgentOps.


MAKE AN APP CHAT WITH STREAMLIT

In [ ]:
!pip install conda